In [2]:
%pip install pandas numpy scikit-learn scikit-image matplotlib tensorflow torch torchvision ultralytics


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
#Load dependencies
import pandas as pd
import numpy as np
import os
import shutil
import base64
import cv2

from ultralytics import YOLO

In [30]:
DATA_DIRECTORY = 'data/images'

LIST_OF_CATEGORIES = [os.path.join(DATA_DIRECTORY, dir) for dir in os.listdir(DATA_DIRECTORY)]

# print(LIST_OF_CATEGORIES)

combined_df = pd.DataFrame()
image_paths = []
image_names = []
image_categories = []

for category in LIST_OF_CATEGORIES:
    for image in os.listdir(category):
        image_paths.append(os.path.join(category, image))
        image_names.append(image)
        image_categories.append(category.split('\\')[-1])

combined_df['image_path'] = image_paths
combined_df['image_name'] = image_names
combined_df['image_category'] = image_categories

combined_df = combined_df.sample(frac=1).reset_index(drop=True)

# combined_df

test_data_size = int(len(combined_df) * 0.15)

test_df = combined_df.head(test_data_size)
combined_df = combined_df.iloc[test_data_size:]

validation_df = combined_df.head(test_data_size)
train_df = combined_df.iloc[test_data_size:]

train_df

,image_path,image_name,image_category
208,data/images\full_garbage\1-216.jpeg,1-216.jpeg,full_garbage
209,data/images\empty_garbage\0-296.jpeg,0-296.jpeg,empty_garbage
210,data/images\supercharger\19.jpg,19.jpg,supercharger
211,data/images\empty_garbage\0-396.jpg,0-396.jpg,empty_garbage
212,data/images\full_garbage\1-269.jpeg,1-269.jpeg,full_garbage
...,...,...,...
691,data/images\full_garbage\1-22.jpg,1-22.jpg,full_garbage
692,data/images\empty_garbage\0-40.jpg,0-40.jpg,empty_garbage
693,data/images\full_garbage\1-631.png,1-631.png,full_garbage
694,data/images\full_garbage\1-521.jpg,1-521.jpg,full_garbage


In [31]:
## Organizing data for yolo model

TRAINING_DATA_PATH = 'data/classification/train/'
VALIDATION_DATA_PATH = 'data/classification/val/'
TEST_DATA_PATH = 'data/classification/test/'
LIST_OF_DIRECTORIES = [TRAINING_DATA_PATH, VALIDATION_DATA_PATH, TEST_DATA_PATH]
LIST_OF_DATAFRAMES = [train_df, validation_df, test_df]

for dir in LIST_OF_DIRECTORIES:
    index = LIST_OF_DIRECTORIES.index(dir)

    if not os.path.isdir(dir):
        os.makedirs(dir, exist_ok=True)

    current_dataframe = LIST_OF_DATAFRAMES[index]

    for row in current_dataframe.itertuples(index=True):
        directory_name = row.image_category
        image_path = row.image_path
        image_name = row.image_name

        if not os.path.isdir(os.path.join(dir, directory_name)):
            os.makedirs(os.path.join(dir, directory_name), exist_ok=True)

        directory_path = os.path.join(dir, directory_name)

        shutil.copy(image_path, os.path.join(directory_path, image_name))

In [34]:
# Model training

model = YOLO('yolov8n-cls.pt')

model.train(data='data/classification', epochs=200, patience=5, verbose=True, dropout=0.3, imgsz=256, )

New https://pypi.org/project/ultralytics/8.3.28 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.63  Python-3.12.5 torch-2.3.1+cpu CPU (AMD Ryzen 5 6600H with Radeon Graphics)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=data/classification, epochs=200, time=None, patience=5, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fal

train: Scanning C:\Users\Dini\Desktop\WebDev\2024\Unihack-RWS\backend\ai\data\classification\train... 642 images, 0 corrupt: 100%|██████████| 642/642 [00:00<?, ?it/s]
val: Scanning C:\Users\Dini\Desktop\WebDev\2024\Unihack-RWS\backend\ai\data\classification\val... 197 images, 0 corrupt: 100%|██████████| 197/197 [00:00<?, ?it/s]

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)


TensorBoard: model graph visualization added 
Image sizes 256 train, 256 val
Using 0 dataloader workers
Logging results to runs\classify\train4
Starting training for 200 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/200         0G     0.8797          2        256: 100%|██████████| 41/41 [01:14<00:00,  1.82s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:19<00:00,  2.76s/it]

...

     46/200         0G    0.04298          2        256: 100%|██████████| 41/41 [01:11<00:00,  1.76s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:18<00:00,  2.70s/it]

                   all          1          1



      Epoch    GPU_mem       loss  Instances       Size


     47/200         0G    0.01872         16        256:  51%|█████     | 21/41 [00:39<00:37,  1.88s/it]


KeyboardInterrupt: 

In [5]:
# Loading already trained model

model = YOLO('ImageClassifier/best.pt')

In [6]:
# Validate the model

validation_results = model.val()
validation_results.top1
validation_results.top5

Ultralytics YOLOv8.2.63  Python-3.12.5 torch-2.3.1+cpu CPU (AMD Ryzen 5 6600H with Radeon Graphics)
YOLOv8n-cls summary (fused): 73 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs

Dataset not found , missing path C:\Users\Dini\Desktop\WebDev\2024\AI-Behavioral-Biometrics-Authentication\datasets\data\classification, attempting download...
 Download failure, retrying 1/3 https://ultralytics.com/assets/data/classification.zip...
Dataset download success  (3.5s), saved to C:\Users\Dini\Desktop\WebDev\2024\AI-Behavioral-Biometrics-Authentication\datasets\data\classification

WARNING  Dataset 'split=val' not found, using 'split=test' instead.


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Dini\\Desktop\\WebDev\\2024\\AI-Behavioral-Biometrics-Authentication\\datasets\\data\\classification\\train'

In [19]:
# Predictions

list_of_predicted_results = []
list_of_actual_results = []

for file in os.listdir(TEST_DATA_PATH):
    results = model(os.path.join(TEST_DATA_PATH, file))
    for result in results:
        list_of_predicted_results.append(result)
        list_of_actual_results.append(file)


image 1/45 c:\Users\Dini\Desktop\WebDev\2024\Unihack-RWS\backend\ai\data\classification\test\empty_garbage\0-105.jpeg: 256x256 empty_garbage 1.00, full_garbage 0.00, supercharger 0.00, 14.5ms
image 2/45 c:\Users\Dini\Desktop\WebDev\2024\Unihack-RWS\backend\ai\data\classification\test\empty_garbage\0-107.jpeg: 256x256 empty_garbage 1.00, full_garbage 0.00, supercharger 0.00, 18.1ms
image 3/45 c:\Users\Dini\Desktop\WebDev\2024\Unihack-RWS\backend\ai\data\classification\test\empty_garbage\0-108.jpeg: 256x256 empty_garbage 1.00, full_garbage 0.00, supercharger 0.00, 18.0ms
image 4/45 c:\Users\Dini\Desktop\WebDev\2024\Unihack-RWS\backend\ai\data\classification\test\empty_garbage\0-117.jpeg: 256x256 empty_garbage 1.00, full_garbage 0.00, supercharger 0.00, 18.6ms
image 5/45 c:\Users\Dini\Desktop\WebDev\2024\Unihack-RWS\backend\ai\data\classification\test\empty_garbage\0-165.jpeg: 256x256 empty_garbage 1.00, full_garbage 0.00, supercharger 0.00, 22.1ms
image 6/45 c:\Users\Dini\Desktop\WebDev

In [20]:
# prediction accuracy

correct_answers = []
index = 0
for result in list_of_predicted_results:
    top1_person = result.probs.top1
    top1_person_name = result.names[top1_person]

    if top1_person_name == list_of_actual_results[index]:
        correct_answers.append(1)

    index += 1

accuracy = len(correct_answers) / len(list_of_predicted_results)

print(accuracy)

1.0


In [29]:
model = YOLO('best.pt')

# Decode the base64 string to a numpy array
image_data = base64.b64decode("data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/7QCcUGhvdG9zaG9wIDMuMAA4QklNBAQAAAAAAIAcAmcAFHRVd0hwdmZzWmh5RFZmV0p5UmtlHAIoAGJGQk1EMGEwMDBhODcwMTAwMDBmZTAxMDAwMDhhMDIwMDAwY2MwMjAwMDAxMjAzMDAwMDliMDMwMDAwMmYwNDAwMDA2NTA0MDAwMGFiMDQwMDAwZjYwNDAwMDBlZTA1MDAwMP/bAEMABgQFBgUEBgYFBgcHBggKEAoKCQkKFA4PDBAXFBgYFxQWFhodJR8aGyMcFhYgLCAjJicpKikZHy0wLSgwJSgpKP/bAEMBBwcHCggKEwoKEygaFhooKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKP/CABEIADIAMgMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAABQQGAQMHAgj/xAAYAQEBAQEBAAAAAAAAAAAAAAADAgABBP/aAAwDAQACEAMQAAABXRIMwmk+d7xYUa3CudYC2FHxJzX2Aep5mCnzWDSidp4+tHgQuCyVmk1eQVvvvZT6rWGh7aZJr5tyHdnYFTh+G52oA0//xAAjEAABAwQBBAMAAAAAAAAAAAABAAIDBBESMjETISIjFDRB/9oACAEBAAEFAjJZF5cmhGVrVyCFDB6c8jGLprHGSko+rNVMDJH8QQWhgNn3F8/GGq6EteyTPzDGHwCikbg21qt6paqSF88+Ybq2xXaNvyolI9jz61Slhlbrliivw6qi71DNZNzwdDqVQfZHH//EAB0RAAICAwADAAAAAAAAAAAAAAERAAIDECEEMTL/2gAIAQMBAT8BcMcZmEgW7PNNSHKiqejZHkyZrZPqNat70df/xAAcEQACAgIDAAAAAAAAAAAAAAABAgAREDEEIFH/2gAIAQIBAT8BC1AJWH1OP5GL2BirioF1CLwNdP/EACcQAAIBAgUDBAMAAAAAAAAAAAABAhEhAxASUXEiMpETQUJhcoGC/9oACAEBAAY/Aj6ytcrlDhFyxLU3pruPSojWWH+KFsVVKLYsd+iprw6yg9heomn9keMtEvKLOpHc6X0v2I71EKjfg1yl/KPm/wBFW5+Dun4IqdZVapYQ0XyWWDyhDEIQuDB5Ef/EACMQAQACAgAFBQEAAAAAAAAAAAEAESExUWFxgZEQIKGx0UH/2gAIAQEAAT8hIWuI7Wox6jFZ+UMPF6Nq1v6pazwmehamf3Ud7KYUlTBUQRNWfRMqq27jdXeILwLCkNkatx3nTBIRndG7SKYhTNkLFX0TsCwVsMrUc1Yk8s+BKCyGqyiG12mhZWXk6P2HjOQf2D/Pg/sO7PiMN9YfGSxFyt71EGlzj5+hqjPC4IZeM9ry2svhZ8af/9oADAMBAAIAAwAAABCxBXNd47HfJ76OKIL/xAAaEQEBAQEBAQEAAAAAAAAAAAABABEhEDFR/9oACAEDAQE/EFrhbOvIG4TtG2ARi7JR4/lkJY6WeLhKwlnifS+Dz//EABwRAQEAAwADAQAAAAAAAAAAAAEAESExEEFRcf/aAAgBAgEBPxAtmD9sjbARXSSLwQE7PtmGMvLdwThs3K9R3x//xAAkEAEAAgIBBAIDAQEAAAAAAAABABEhMUFRcaGxgZFhwdEQ8f/aAAgBAQABPxBjSHM3NwnXvHbCrtBJPPfB8w1ggLphbl85q8JVLjgaIhwRWswno0Wgq+jxBYPF4lY5rhg7FoIAW10jbJgZQEAe9IQWUBVojEulN7rmubv7igrZkYVBhSYHm4FShaum7Di9OszCVoCkcMIF69Eq5lkL50j+TklApywsP06gUDAb4hI6Ac7cnR7QwMLXkoBmI9D6j1lHOX4Y5BDASvzitwKAHRV5YfStuBM/zLHlzvQAlCo3jslyVv1QYei/wao+/uZ+u2fNRsFsMBGd0/2WLqEF3V+4AfT1TzP1GrOWeZGleMHr/Mbv/wCDG7OHqf/Z")
np_array = np.frombuffer(image_data, np.uint8)
image = cv2.imdecode(np_array, cv2.IMREAD_COLOR)

# Run the trained model on the decoded image
results = model(image)

top1 = results[0].probs.top1
top1_name = results[0].names[top1]

print(top1_name == 'empty_garbage')

WARNING  'source' is missing. Using 'source=C:\Compilers\Python\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Compilers\Python\Lib\site-packages\ultralytics\assets\bus.jpg: 256x256 supercharger 0.74, full_garbage 0.18, empty_garbage 0.08, 22.6ms
image 2/2 C:\Compilers\Python\Lib\site-packages\ultralytics\assets\zidane.jpg: 256x256 full_garbage 0.94, empty_garbage 0.05, supercharger 0.01, 21.8ms
Speed: 20.1ms preprocess, 22.2ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 256)
False
